# GAP(Global Average Precision)
Reference:
- [Suspected error in the leaderboard score calculator (solved)](https://www.kaggle.com/c/landmark-recognition-challenge/discussion/52080)

In [1]:
import os
import random
import numpy as np
import pandas as pd

In [2]:
P = []
hits = 0
N = 117703
testset_landmark_ratio = 1.0
M = testset_landmark_ratio * (N - 1)
for i in range(1, N):
  rel = 1.0 * (random.uniform(0.0, 1.0) < 0.5)
  hits = hits + rel
  P.append(rel * hits / i)

sum(P) / M

0.24809426294073644

In [3]:
def comp_landmark(landmark_true, landmark_pred):
    landmark_true = landmark_true.strip()
    landmark_pred = landmark_pred.strip()
    if len(landmark_true) > 0 and len(landmark_pred) > 0:
        pair = ' '.join(filter(lambda x: x, landmark_pred.split(' '))).split(' ')
        if landmark_true == pair[0]:
#             print(landmark_true, pair[0])
            return True, float(pair[1])
    return False, 0.

print(comp_landmark('', ''))
print(comp_landmark(' ', ' '))
print(comp_landmark('   ', '   '))
print(comp_landmark('0012', '0012 0.2'))
print(comp_landmark('   0012   ', '   0012   0.2   '))
print(comp_landmark('   0012   ', '   0013   0.2   '))

(False, 0.0)
(False, 0.0)
(False, 0.0)
(True, 0.2)
(True, 0.2)
(False, 0.0)


In [4]:
import numpy as np
import pandas as pd


            
def gap(y_true, y_pred):
    if len(y_true) != len(y_pred):
        raise Exception('len(y_true) and len(y_pred) should be equal!')
        
    # Convert y_pred pandas.DataFrame to dict
    y_pred_dict = {}
    for i in range(len(y_pred)):
        y_pred_dict[y_pred['id'][i]] = y_pred['landmarks'][i]
    print(y_pred_dict)
    
    n = len(y_true)
    m = 0.
    n_correct = 0
    p_arr = []
    for i in range(n):
#         print('*' * 20)
        y_true_landmark = y_true['landmarks'][i]
        if y_true['id'][i] not in y_pred_dict:
            raise Exception('Can not find id in y_pred: %s' % y_true['id'][i])
        y_pred_landmarks_pair = y_pred_dict[y_true['id'][i]]
#         print(y_true_landmark, y_pred_landmarks_pair)
        is_equal, rel = comp_landmark(y_true_landmark, y_pred_landmarks_pair)
#         print('is_equal, rel: ', is_equal, rel)
        if is_equal:
            m = m + 1.
            p = m / (i + 1)
        else:
            p = 0.
        p_arr.append(p * rel)
    print('p_arr', p_arr)
    return sum(p_arr) / m

In [5]:
data_true = {
    'id': [
        'a1', 
        'a2', 
        'a3',
        'a4', 
        'a5', 
        'a6'
    ],
    'landmarks': [
        '1',
        '15',
        '',
        '',
        '11',
        '1'
    ]
}

data_pred = {
    'id': [
        'a1', 
        'a2', 
        'a3',
        'a4', 
        'a5', 
        'a6'
    ],
    'landmarks': [
        '1 1',
        '15 1',
        '',
        '',
        '11 1',
        '1 1'
    ]
}

y_true = pd.DataFrame(data_true)
y_pred = pd.DataFrame(data_pred)
display(y_true)
display(y_pred)
print(gap(y_true, y_pred))

,id,landmarks
0,a1,1
1,a2,15
2,a3,
3,a4,
4,a5,11
5,a6,1


,id,landmarks
0,a1,1 1
1,a2,15 1
2,a3,
3,a4,
4,a5,11 1
5,a6,1 1


{'a1': '1 1', 'a2': '15 1', 'a3': '', 'a4': '', 'a5': '11 1', 'a6': '1 1'}
p_arr [1.0, 1.0, 0.0, 0.0, 0.6, 0.6666666666666666]
0.8166666666666667


In [6]:
## Try to proof: use less than 1.0 probability will reduce the gap score, should all use 1.0 as predict probability
data_pred = {
    'id': [
        'a1', 
        'a2', 
        'a3',
        'a4', 
        'a5', 
        'a6'
    ],
    'landmarks': [
        '1 0.5',
        '15 0.5',
        '',
        '',
        '11 0.5',
        '1 0.5'
    ]
}
y_pred = pd.DataFrame(data_pred)
print(gap(y_true, y_pred))

{'a1': '1 0.5', 'a2': '15 0.5', 'a3': '', 'a4': '', 'a5': '11 0.5', 'a6': '1 0.5'}
p_arr [0.5, 0.5, 0.0, 0.0, 0.3, 0.3333333333333333]
0.4083333333333333


In [7]:
# Try to proof: predict some thing at empty y_true landmark will not reduce the gap score
data_pred = {
    'id': [
        'a1', 
        'a2', 
        'a3',
        'a4', 
        'a5', 
        'a6'
    ],
    'landmarks': [
        '1 1',
        '15 1',
        '',
        '12 1',
        '11 1',
        '1 1'
    ]
}
y_pred = pd.DataFrame(data_pred)
print(gap(y_true, y_pred))

{'a1': '1 1', 'a2': '15 1', 'a3': '', 'a4': '12 1', 'a5': '11 1', 'a6': '1 1'}
p_arr [1.0, 1.0, 0.0, 0.0, 0.6, 0.6666666666666666]
0.8166666666666667
